In [1]:
!pip install opencv-python-headless
!pip install torch torchvision


In [2]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 781, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 781 (delta 280), reused 277 (delta 228), pack-reused 374
Receiving objects: 100% (781/781), 3.30 MiB | 26.59 MiB/s, done.
Resolving deltas: 100% (325/325), done.


In [3]:
%cd yolov9
!pip install -r requirements.txt

/kaggle/working/yolov9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 9.3 MB/s eta 0:00:00:00:01


In [4]:
with open('/kaggle/working/yolov9/data/monkey.yaml', 'w') as f:
    f.write("""
train: /kaggle/input/mk-sh-dataset/train/images
val: /kaggle/input/mk-sh-dataset/valid/images
test: /kaggle/input/mk-sh-dataset/test/images

nc: 1
names: ['Monkey']
    """)


In [5]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt


In [6]:
!python train.py \
--batch 16 --epochs 25 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /kaggle/working/yolov9/data/monkey.yaml \
--weights weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml

2024-07-22 12:16:43.353070: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 12:16:43.353169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 12:16:43.506233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=weights/gelan-c.pt, cfg=models/detect/gelan-c.yaml, data=/kaggle/working/yolov9/data/monkey.yaml, hyp=hyp.scratch-high.yaml, epochs=25, batch_size=16, i

In [7]:
!python val.py \
--img 640 --batch 32 --conf 0.001 --iou 0.7 --device 0 \
--data data/monkey.yaml \
--weights runs/train/exp/weights/best.pt

val: data=data/monkey.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
gelan-c summary: 387 layers, 25227859 parameters, 0 gradients, 101.8 GFLOPs
val: Scanning /kaggle/input/mk-sh-dataset/valid/labels... 337 images, 0 backgrou
val: WARNING ⚠️ Cache directory /kaggle/input/mk-sh-dataset/valid is not writeable: [Errno 30] Read-only file system: '/kaggle/input/mk-sh-dataset/valid/labels.cache.npy'
                 Class     Images  Instances          P          R      mAP50   
                   all        337        520      0.817      0.762      0.837      0.568
Speed: 0.2ms pre-pro

In [8]:
!python detect.py \
--img 1280 --conf 0.1 --device 0 \
--weights runs/train/exp/weights/best.pt \
--source /kaggle/input/mk-sh-dataset/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/kaggle/input/mk-sh-dataset/test/images, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
gelan-c summary: 387 layers, 25227859 parameters, 0 gradients, 101.8 GFLOPs
image 1/114 /kaggle/input/mk-sh-dataset/test/images/003ce901008b4cb5_jpg.rf.75e48c9ce1ae4d4478fb9315da6e4565.jpg: 1280x1280 2 Monkeys, 70.3ms
image 2/114 /kaggle/input/mk-sh-dataset/test/images/005524e1e5ebadaa_jpg.rf.a4ea11d61c96b0db50be2229be83f36c.jpg: 1280x1280 3 Monkeys, 65.1ms
image 3/114 /kaggle/input/mk-sh-